In [1]:
import glob
import json
import math
import pandas as pd
import numpy as np

In [2]:
# files = glob.glob('../Decision_tree/data_sample/*')
files = glob.glob('../hadian_R_D/*/*.json',recursive = True)
raw_data = []
data=[]

In [3]:
for hit in files:
     with open(hit) as f:
            raw_data.append(json.load(f))

In [4]:
len(raw_data)

583281

In [5]:
def stdev_N_1(list):
    leng = len(list)
    if leng==1:
        leng=2
    m = mean(list)
    s_2=0
    for num in list:
         s_2 += (num - m)**2
    s= math.sqrt(s_2/(leng -1))
    return s

In [6]:
def mean(list):
    try :
        avg = sum(list) / len(list)
        return avg
    except:
        return -1

In [7]:
def Max(list):
    try:
        return max(list)
    except:
        return -1

In [8]:
def Min(list):
    try:
        return min(list)
    except:
        return -1

In [9]:
def Features(dic):
    """
    making featurs
    input : output of Tshark command's as dict
    :return: add fearturs key to dict
    """
    #    variable for calculating featurse
    send_packet_duration, send_packet_size, receive_packet_duration, receive_packet_size = [], [], [], []
    send_time, receive_time = [], []

    #   variable for calculating sequence of packets
    no_send_packet, no_receive_packet, no_packet = 0, 0, 0
    send_seq, receive_seq = 0, 0
    sequence_send_packet, sequence_receive_packet = [], []
   
    for val in dic['packet_list']:
        no_packet += 1
        if val['direction'] == 0:  # send
            if val['payload_size'] != 0 :
                send_packet_size.append(val['payload_size'])
            if len(send_time)>0:    
                send_packet_duration.append(val['time'] - send_time[-1])
                send_time.append(val['time'])
            else:
                send_time.append(val['time'])
            

            if no_packet - no_send_packet == 1:
                send_seq += 1
            else:
                sequence_send_packet.append(send_seq)
                send_seq = 1
            no_send_packet = no_packet
        if val['direction'] == 1:  # recieve
            if val['payload_size'] != 0:
                receive_packet_size.append(val['payload_size'])
            if len(receive_time)>0:    
                receive_packet_duration.append(val['time'] - receive_time[-1])
                receive_time.append(val['time'])
            else:
                receive_time.append(val['time'])    
                
            



            if no_packet - no_receive_packet == 1:
                receive_seq += 1
            else:
                sequence_receive_packet.append(receive_seq)
                receive_seq = 1
            no_receive_packet = no_packet
    sequence_send_packet.append(send_seq)
    sequence_receive_packet.append(receive_seq)

    dic['features'] = { "tag":dic['five_tuple']['tag'],
                        "port" : dic['five_tuple']['dst_port'],
                        "l4proto" : dic['five_tuple']['l4proto'],
                        "l7proto" : dic['five_tuple']['l7proto'],
                        "avg_receive_packet_duration": mean(receive_packet_duration),
                        "avg_receive_packet_size":  mean(receive_packet_size),
                        "avg_send_packet_duration": mean(send_packet_duration),
                        "avg_send_packet_size": mean(send_packet_size),
                        "max_receive_packet_duration": Max(receive_packet_duration),
                        "max_receive_packet_size": Max(receive_packet_size),
                        "max_receive_sequence_packet_count":Max(sequence_receive_packet),
                        "max_send_packet_duration":  Max(send_packet_duration),
                        "max_send_packet_size": Max(send_packet_size),
                        "max_send_sequence_packet_count": Max(sequence_send_packet),
                        "min_receive_packet_duration": Min(receive_packet_duration),
                        "min_receive_packet_size": Min(receive_packet_size),
                        "min_send_packet_duration": Min(send_packet_duration),
                        "min_send_packet_size": Min(send_packet_size),
                        "stddev_receive_packet_duration": stdev_N_1(receive_packet_duration),
                        "stddev_receive_packet_size": stdev_N_1(receive_packet_size),
                        "stddev_send_packet_duration": stdev_N_1(send_packet_duration),
                        "stddev_send_packet_size": stdev_N_1(send_packet_size)}

    return dic['features']

In [10]:
for d in raw_data:
    data.append(Features(d))

In [11]:
df = pd.DataFrame(data)

In [12]:
df.head()

,tag,port,l4proto,l7proto,avg_receive_packet_duration,avg_receive_packet_size,avg_send_packet_duration,avg_send_packet_size,max_receive_packet_duration,max_receive_packet_size,...,max_send_packet_size,max_send_sequence_packet_count,min_receive_packet_duration,min_receive_packet_size,min_send_packet_duration,min_send_packet_size,stddev_receive_packet_duration,stddev_receive_packet_size,stddev_send_packet_duration,stddev_send_packet_size
0,None,80,6,HTTP,333.000000,2.000000,266.600000,-1.000000,1058,2,...,-1,2,4,2,1,-1,489.558304,0.000000,488.100195,-0.000000
1,None,443,6,TLS,105.500000,994.769231,125.000000,172.000000,616,1398,...,291,10,0,32,0,33,185.555742,577.453195,257.188890,130.157597
2,None,80,6,HTTP,27.000000,-1.000000,28.250000,-1.000000,80,-1,...,-1,2,0,-1,0,-1,45.902070,-0.000000,22.396056,-0.000000
3,None,443,6,TLS,5113.214286,831.888889,844.272727,310.333333,62298,1388,...,517,6,0,32,0,33,16485.849103,636.195222,1273.025851,229.676874
4,None,443,6,TLS,161.428571,266.333333,161.000000,585.333333,411,730,...,1211,2,0,24,0,28,141.776652,401.684370,175.632734,594.452970


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583281 entries, 0 to 583280
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   tag                                32416 non-null   object 
 1   port                               583281 non-null  int64  
 2   l4proto                            583281 non-null  int64  
 3   l7proto                            583281 non-null  object 
 4   avg_receive_packet_duration        583281 non-null  float64
 5   avg_receive_packet_size            583281 non-null  float64
 6   avg_send_packet_duration           583281 non-null  float64
 7   avg_send_packet_size               583281 non-null  float64
 8   max_receive_packet_duration        583281 non-null  int64  
 9   max_receive_packet_size            583281 non-null  int64  
 10  max_receive_sequence_packet_count  583281 non-null  int64  
 11  max_send_packet_duration           5832

In [14]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.matshow(df.corr())
# plt.show()

In [15]:
# df.corr()

In [16]:
# df['port'] = df['port'].astype('str')
# df['port'] =  df['port'].astype('category')
# df['l4proto'] = df['l4proto'].astype('str')
df['l4proto'] = df['l4proto'].astype('category')
df['l7proto'] = df['l7proto'].astype('category')

In [17]:
# df['l7proto']=df['l7proto'].where(df['l7proto'].notnull(), None)
df['l7proto'] = df['l7proto'].replace([''], ['null'])
df['tag'] = df['tag'].replace([None], ['None'])

In [18]:
df['l7proto'].unique()

array(['HTTP', 'TLS', 'null', 'DNS', 'ESP', 'L2TP', 'PPTP'], dtype=object)

In [19]:
df['tag'].unique()

array(['None', 'instagram'], dtype=object)

In [20]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
# df['port_cat'] = labelencoder.fit_transform(df['port'])
df['l4proto_cat'] = labelencoder.fit_transform(df['l4proto'])
df['l7proto_cat'] = labelencoder.fit_transform(df['l7proto'])

In [21]:
from sklearn.preprocessing import OneHotEncoder 
enc = OneHotEncoder(handle_unknown='ignore')
# enc_port = pd.DataFrame(enc.fit_transform(df[['port_cat','l4proto_cat','l7proto_cat']]).toarray())
enc_port = pd.DataFrame(enc.fit_transform(df[['l4proto_cat','l7proto_cat']]).toarray())
# enc_port = pd.DataFrame(enc.fit_transform(df[['l7proto_cat']]).toarray())
df = df.join(enc_port)

In [22]:
# df.drop('port_cat',inplace=True,axis=1)
df.drop('l4proto_cat',inplace=True,axis=1)
df.drop('l7proto_cat',inplace=True,axis=1)
df.drop('l4proto',inplace=True,axis=1)
df.drop('l7proto',inplace=True,axis=1)

In [23]:
df.head()

,tag,port,avg_receive_packet_duration,avg_receive_packet_size,avg_send_packet_duration,avg_send_packet_size,max_receive_packet_duration,max_receive_packet_size,max_receive_sequence_packet_count,max_send_packet_duration,...,stddev_send_packet_size,0,1,2,3,4,5,6,7,8
0,None,80,333.000000,2.000000,266.600000,-1.000000,1058,2,2,1136,...,-0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,None,443,105.500000,994.769231,125.000000,172.000000,616,1398,11,989,...,130.157597,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,None,80,27.000000,-1.000000,28.250000,-1.000000,80,-1,3,54,...,-0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,None,443,5113.214286,831.888889,844.272727,310.333333,62298,1388,4,4002,...,229.676874,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,None,443,161.428571,266.333333,161.000000,585.333333,411,730,2,549,...,594.452970,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
df['tag'].value_counts()

None         550865
instagram     32416
Name: tag, dtype: int64

In [25]:
array = df.values
X = array[:,1:]
Y = array[:,0]

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [27]:
rescaledX = scaler.fit_transform(X)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split( 
    rescaledX, Y, test_size = 0.3, random_state = 100)

In [30]:
from sklearn.ensemble import RandomForestClassifier

In [65]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42,min_samples_leaf=4)

In [ ]:
rf.fit(X_train, y_train);
y_pred = rf.predict(X_test)

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
confusion_matrix(y_test, y_pred)

array([[165071,    366],
       [  1444,   8104]])

In [34]:
 accuracy_score(y_test,y_pred)*100

98.96562562505358

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        None       0.99      1.00      0.99    165437
   instagram       0.96      0.85      0.90      9548

    accuracy                           0.99    174985
   macro avg       0.97      0.92      0.95    174985
weighted avg       0.99      0.99      0.99    174985



## resampling


In [36]:
from sklearn.utils import resample

In [37]:
df_majority = df[df['tag']=='None']
df_minority_unsampled = df[df['tag']=='instagram']

In [38]:
df_majority_downsample = resample(df_majority, 
                                 replace=True,     
                                 n_samples=33000,    
                                 random_state=123)

In [39]:
df_resampled = pd.concat([df_majority_downsample, df_minority_unsampled])

In [40]:
df_resampled

,tag,port,avg_receive_packet_duration,avg_receive_packet_size,avg_send_packet_duration,avg_send_packet_size,max_receive_packet_duration,max_receive_packet_size,max_receive_sequence_packet_count,max_send_packet_duration,...,stddev_send_packet_size,0,1,2,3,4,5,6,7,8
294227,None,443,2338.200000,880.100000,2389.600000,286.500000,29806,1388,4,29607,...,122.558441,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
29696,None,443,65.250000,1029.800000,75.900000,240.666667,271,1346,3,237,...,242.079188,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
18779,None,80,213.000000,7.000000,234.000000,524.000000,425,7,2,277,...,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
203938,None,443,655.776119,132.000000,543.037037,1305.324324,26263,185,7,26125,...,289.876171,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
205839,None,53,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1,1,-1,...,-0.000000,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583236,instagram,443,23.942857,1186.562500,26.545455,264.333333,253,1380,14,518,...,314.948144,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
583246,instagram,443,532.375000,897.090909,570.400000,213.250000,8342,1380,9,8288,...,283.214142,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
583262,instagram,443,9.235294,801.166667,38.666667,272.857143,65,1232,12,72,...,448.725762,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
583263,instagram,443,411.250000,1091.062016,2632.200000,290.047619,51507,1232,34,51496,...,418.794637,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [41]:
resampled_array = df_resampled.values
resampled_X = resampled_array[:,4:]
resampled_Y = resampled_array[:,0]

In [42]:
rescaled_resampled_X = scaler.fit_transform(resampled_X)

In [43]:
X_train, X_test, y_train, y_test = train_test_split( 
    rescaled_resampled_X, resampled_Y, test_size = 0.3, random_state = 100)

In [44]:
rf_resample = RandomForestClassifier(n_estimators = 1000, random_state = 42,min_samples_leaf=4)

In [45]:
rf_resample.fit(X_train, y_train);
y_pred = rf_resample.predict(X_test)

In [46]:
confusion_matrix(y_test, y_pred)

array([[9504,  348],
       [ 310, 9463]])

In [47]:
 accuracy_score(y_test,y_pred)*100

96.64713375796178

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        None       0.97      0.96      0.97      9852
   instagram       0.96      0.97      0.97      9773

    accuracy                           0.97     19625
   macro avg       0.97      0.97      0.97     19625
weighted avg       0.97      0.97      0.97     19625



In [53]:
estimator = rf.estimators_[500]
from sklearn.tree import export_graphviz

export_graphviz(
estimator,
out_file='tree.dot',
feature_names=['port','avg_receive_packet_duration', 'avg_receive_packet_size',
       'avg_send_packet_duration', 'avg_send_packet_size',
       'max_receive_packet_duration', 'max_receive_packet_size',
       'max_receive_sequence_packet_count', 'max_send_packet_duration',
       'max_send_packet_size', 'max_send_sequence_packet_count',
       'min_receive_packet_duration', 'min_receive_packet_size',
       'min_send_packet_duration', 'min_send_packet_size',
       'stddev_receive_packet_duration', 'stddev_receive_packet_size',
       'stddev_send_packet_duration', 'stddev_send_packet_size','0','1','2','3','4','5','6','7','8'],
class_names=['ins','null'],
rounded=True,
filled=True
)

### KNN algo

In [54]:
df = pd.DataFrame(data)

In [55]:
df['l4proto'] = df['l4proto'].astype('category')
df['l7proto'] = df['l7proto'].astype('category')

In [56]:
df['l7proto'] = df['l7proto'].replace([''], ['null'])
df['tag'] = df['tag'].replace([None], ['None'])

In [57]:
df['l4proto_cat'] = labelencoder.fit_transform(df['l4proto'])
df['l7proto_cat'] = labelencoder.fit_transform(df['l7proto'])

In [58]:
from sklearn.preprocessing import OneHotEncoder 
enc = OneHotEncoder(handle_unknown='ignore')
enc_port = pd.DataFrame(enc.fit_transform(df[['l4proto_cat','l7proto_cat']]).toarray())
df = df.join(enc_port)

In [59]:
df.drop('l4proto_cat',inplace=True,axis=1)
df.drop('l7proto_cat',inplace=True,axis=1)
df.drop('l4proto',inplace=True,axis=1)
df.drop('l7proto',inplace=True,axis=1)

In [60]:
array = df.values
X = array[:,1:]
Y = array[:,0]

In [61]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)

In [62]:
X_train, X_test, y_train, y_test = train_test_split( 
    rescaledX, Y, test_size = 0.3, random_state = 100)

In [63]:
from sklearn.neighbors import KNeighborsClassifier
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [64]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        None       0.99      0.99      0.99    165437
   instagram       0.89      0.79      0.84      9548

    accuracy                           0.98    174985
   macro avg       0.94      0.89      0.91    174985
weighted avg       0.98      0.98      0.98    174985

